# Running PyTorch vs TensorFlow on GPUs

In [1]:
import numpy as np 
import tensorflow as tf 
import torch
import time

## Summary for TensorFlow

If a TensorFlow operation has both CPU and GPU implementation, the GPU devices will be given priority, unless specified otherwise. 

e.g., on a system with devices `cpu:0` and `gpu:0`, `gpu:0` will automatically be selected to run `matmul`.

Below are three examples:

1. Generate matrices *and* run `matmul` all on the CPU.

2. Run `matmul` on a single GPU, but with matrix generation done on CPU (ie. with numpy). Note that this is not recommended, as the overhead to send data to the GPU to be processed at each iteration does not provide any speedup from the case where everything is run on the CPU.

3. Generate tensors that are automatically assigned to the GPU memory (ie. the default for TensorFlow) and then run `matmul` on these tensors. In this case everything will be done on the GPU and speedup will be considerable.


In [24]:
# Example 1 (everything done on CPU)

N = 100
ntimes = 1000
tstart = time.time()

for _ in range(ntimes):
    with tf.device('/cpu:0'):
        a = tf.random.normal([N,N])
        b = tf.random.normal([N,N])
        c = tf.matmul(a,b)

tend = time.time()
print('Time Elapsed: {} s\n'.format(tend-tstart))


Time Elapsed: 0.8208062648773193 s



In [25]:
# Example 2 (matrices generated on CPU, multiplication done on GPU)

N = 100
ntimes = 1000
tstart = time.time()

for _ in range(ntimes):
    with tf.device('/cpu:0'):
        a = tf.random.normal([N,N])
        b = tf.random.normal([N,N])
    c = tf.matmul(a,b)

tend = time.time()
print('Time Elapsed: {} s\n'.format(tend-tstart))


Time Elapsed: 1.0858714580535889 s



In [23]:
# Example 3 (everything done on GPU)

N = 100
ntimes = 1000
tstart = time.time()

for _ in range(ntimes):
    a = tf.random.normal([N,N])
    b = tf.random.normal([N,N])
    c = tf.matmul(a,b)

tend = time.time()
print('Time Elapsed: {} s\n'.format(tend-tstart))


Time Elapsed: 0.2559967041015625 s



## Summary for PyTorch

To utilise the GPU implementation, you must first manually allocate tensors in GPU memory (unlike TensorFlow, PyTorch does not do this automatically). Afterwards, when you perform operations on those tensors, e.g. `matmul`, they will be run on the GPU.

To allocate tensors in GPU memory, you must make them of a `torch.cuda` tensor type.

1. Generate matrices *and* run `matmul` all on the CPU. This is essentially the PyTorch default, even when using the built-in functions for tensor generation.

2. Run `matmul` on a single GPU, but with matrix generation done on CPU. Note that you must manually ensure that the `matmul` operation is being done on the GPU. 

3. Manually assign tensors that to the GPU memory and then run `matmul` on these tensors. Note that once the tensors are assigned to the GPU, all further operations done on these tensors will be performed on the GPU *without* having to specify.

In [18]:
# Example 1 (everything done on CPU)

N = 100
ntimes = 1000
tstart = time.time()

for _ in range(ntimes):
    a = torch.randn(N,N)
    b = torch.randn(N,N)
    c = torch.matmul(a,b)

tend = time.time()
print('Time Elapsed: {} s\n'.format(tend-tstart))


Time Elapsed: 1.008932113647461 s



In [19]:
# Example 2 (matrices generated on CPU, multiplication done on GPU)

N = 100
ntimes = 1000
tstart = time.time()
cuda = torch.device('cuda')

for _ in range(ntimes):
    a = torch.randn(N,N)
    b = torch.randn(N,N)
    c = torch.matmul(a,b).to(cuda)

tend = time.time()
print('Time Elapsed: {} s\n'.format(tend-tstart))


Time Elapsed: 0.25334668159484863 s



In [20]:
# Example 3 (everything done on GPU)

N = 100
ntimes = 1000
tstart = time.time()

for _ in range(ntimes):
    a = torch.randn(N,N, device='cuda:0')
    b = torch.randn(N,N, device='cuda:0')
    c = torch.matmul(a,b)

tend = time.time()
print('Time Elapsed: {} s\n'.format(tend-tstart))

Time Elapsed: 0.03806352615356445 s

